In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from labserverapps.predictors.mhc_peptides import PeptideAffinityPredictions
from IPython.display import display, Markdown
import plotly.figure_factory as ff
from scipy.spatial.distance import pdist, squareform
import seaborn as sns
from datetime import datetime
from pathlib import Path
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt

In [ ]:
peptide_file = '/home/labcaron/Projects/LabWebApp/example.csv'
pep_header = 'Peptide'
alleles = 'HLA-A03:02,HLA-A02:02'
min_length = 8
max_length = 14
pep_file_delimiter = ','
experiment_name = 'unnamed'
submitter = None

In [ ]:
p = PeptideAffinityPredictions(peptide_file=peptide_file,
                               pep_header=pep_header,
                               pep_file_delimiter=pep_file_delimiter,
                               alleles=alleles,
                               min_length=min_length,
                               max_length=max_length)

In [ ]:
p.make_netmhcpan4_predictions()

In [ ]:
p.annotate_binding_predictions()
f = Path(peptide_file)
p.annotated_predictions.to_csv(str(f.parent/f.stem)+'_netMHCpan_results.csv', index=False)

In [ ]:
n_peps = len(p.peptides)
n_unique_peps = len(p.annotated_predictions['Peptide']['Peptide'].unique())
lengths = np.vectorize(len)(p.peptides)
unique_lengths = np.vectorize(len)(p.annotated_predictions['Peptide']['Peptide'].unique())
total_within_lengths = np.sum((lengths >= min_length)*(lengths <= max_length))
unique_within_lengths = np.sum((unique_lengths >= min_length)*(unique_lengths <= max_length))

annotation = p.annotated_predictions['Annotation']
peps = p.annotated_predictions['Peptide']

total_n_strong = len(peps.loc[annotation['Binding Strength']=='Strong', 'Peptide'].values)
unique_n_strong = len(peps.loc[annotation['Binding Strength']=='Strong', 'Peptide'].unique())

total_n_medium = len(peps.loc[annotation['Binding Strength']=='Medium', 'Peptide'].values)
unique_n_medium = len(peps.loc[annotation['Binding Strength']=='Medium', 'Peptide'].unique())

total_n_weak = len(peps.loc[annotation['Binding Strength']=='Weak', 'Peptide'].values)
unique_n_weak = len(peps.loc[annotation['Binding Strength']=='Weak', 'Peptide'].unique())

In [ ]:
%%html
<style>
  table {margin-left: 5pt !important;}
</style>

In [ ]:
Markdown(f'''
# MHC-associated peptides report

Experiment: {experiment_name}  
Date: {str(datetime.date(datetime.now()))}  
File: {peptide_file}  
Submitted by: {submitter}

### Notes on allele annotation
Annotation of peptides with alleles is according to percent rank as reported by netMHCpan. Peptides are simply
annotated with the allele with the best rank. Rank <= 0.5 is considered strong binding, 0.5 < rank <= 2.0
is considered medium binding, and rank > 2.0 is considered weak binding. Even if a peptide has strong binding
for more than one allele, it will be annotated with only one. However, you will be able to see in the heatmaps
when there are clusters of peptides with similar binding strengths for multiple alleles regardless of their
annotation.

## General description
| |Total Peptides|Unique Peptides|
|---|---|---|
|Identified sequences | {n_peps}| {n_unique_peps}|
|Between {min_length} & {max_length} mers (inclusive)| {total_within_lengths}|{unique_within_lengths}|
|Strong binders|{total_n_strong}|{unique_n_strong}|
|Medium binders|{total_n_medium}|{unique_n_medium}|
|Weak binders|{total_n_weak}|{unique_n_weak}|
|% Strong binders|{round((total_n_strong/n_peps)*100, 2)}|{round((unique_n_strong/n_unique_peps)*100, 2)}|
''')

In [ ]:
# here we need to generate an allele-specific table like the above...
# do it with display and Markdown from Ipython
header = '||'+'|'.join([x for x in p.df_alleles])+'|\n'
thing_under_header = '|---|'+''.join(['---|' for x in p.df_alleles])+'\n'

total_unique = '|Total peptides|'+'|'.join([str(len(peps.loc[(annotation['Allele']==a), 'Peptide'].unique()))
                                       for a in p.df_alleles])+'|\n'

unique_strong_binders = '|Strong binders|'+'|'.join([str(len(peps.loc[(annotation['Allele']==a)
                                                &(annotation['Binding Strength']=='Strong'), 'Peptide'].unique()))
                                       for a in p.df_alleles])+'|\n'

unique_medium_binders = '|Medium binders|'+'|'.join([str(len(peps.loc[(annotation['Allele']==a)
                                                &(annotation['Binding Strength']=='Medium'), 'Peptide'].unique()))
                                       for a in p.df_alleles])+'|\n'

unique_weak_binders = '|Weak binders|'+'|'.join([str(len(peps.loc[(annotation['Allele']==a)
                                                &(annotation['Binding Strength']=='Weak'), 'Peptide'].unique()))
                                       for a in p.df_alleles])+'|\n'

percent_strong_binders = '|% Strong binders|'+'|'.join([str(round(
    len(peps.loc[(annotation['Allele']==a)&(annotation['Binding Strength']=='Strong'), 'Peptide'].unique())/
    len(peps.loc[(annotation['Allele']==a), 'Peptide'].unique())*100, 2))
                                       for a in p.df_alleles])+'|'

In [ ]:
display(Markdown('## Allele-specific description'))

desc = 'Counts of unique peptide sequences for strong, medium and weak binders.'

display(Markdown(desc))
display(Markdown(header+
                 thing_under_header+
                 total_unique+
                 unique_strong_binders+
                 unique_medium_binders+
                 unique_weak_binders+
                 percent_strong_binders))

## Figures

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x = p.df_alleles,
    y = [len(peps.loc[annotation['Allele']==allele, 'Peptide'].unique()) for allele in p.df_alleles],
    name = 'All binders'
))
fig.add_trace(go.Bar(
    x = p.df_alleles,
    y = [len(peps.loc[(annotation['Allele']==allele)&(annotation['Binding Strength']=='Strong'), 'Peptide']
             .unique()) for allele in p.df_alleles],
    name = 'Strong binders'
))

fig.update_layout(
    title_text='Number of unique sequences per allele', # title of plot
    xaxis_title_text='Allele', # xaxis label
    yaxis_title_text='Peptide count', # yaxis label
    template='ggplot2'
)
fig.show()

In [ ]:
display(Markdown('## Peptide length distributions'))
desc = 'Histograms of peptide lengths. All distributions are of unique peptide sequences only. ' +\
    'Histograms are generated for all peptides, and then grouped by allele for strong, medium and weak binders.'
display(Markdown(desc))

display(Markdown(f'### All peptides'))
fig = go.Figure()
for allele in p.df_alleles:
    lengths = peps.loc[annotation['Allele']==allele, 'Peptide'].unique()
    if len(lengths) > 0:
        lengths = np.vectorize(len)(lengths)
    else:
        lengths = []
    fig.add_trace(go.Histogram(
        x=lengths,
        name=allele
    ))

fig.update_layout(
    title_text=f'Peptide length distribution, all binders', # title of plot
    xaxis_title_text='Length', # xaxis label
    yaxis_title_text='Count', # yaxis label
    bargap=0.2, # gap between bars of adjacent location coordinates
    bargroupgap=0.1, # gap between bars of the same location coordinates
    template='ggplot2'
)
fig.show()

for strength in ['Strong', 'Medium', 'Weak']:
    if strength == 'Strong':
        info_str = '% rank <= 0.5'
    elif strength == 'Medium':
        info_str = '0.5 < %rank <= 2.0'
    else:
        info_str = '% rank > 2.0'
    display(Markdown(f'### {strength} binders ({info_str})'))
    fig = go.Figure()
    all_lengths = []
    for allele in p.df_alleles:
        lengths = peps.loc[(annotation['Allele']==allele)&
                           (annotation['Binding Strength']==strength), 'Peptide'].unique()
        if len(lengths) > 0:
            lengths = np.vectorize(len)(lengths)
        else:
            lengths = []
        all_lengths.append(lengths)
        fig.add_trace(go.Histogram(
            x=lengths,
            name=allele
        ))

    fig.update_layout(
        title_text=f'Peptide length distribution, {strength.lower()} binders', # title of plot
        xaxis_title_text='Length', # xaxis label
        yaxis_title_text='Count', # yaxis label
        bargap=0.2, # gap between bars of adjacent location coordinates
        bargroupgap=0.1, # gap between bars of the same location coordinates
        template='ggplot2'
    )
    if np.sum([len(x) for x in all_lengths]) == 0:
        display(Markdown(f'No {strength.lower()} binders'))
        continue
    else:
        fig.show()

## Heatmaps
Heatmaps of ordered binding affinity ranks. Ranks for every allele are used, regardless of annotation.

The ranks are adjusted such that all ranks above 2.5 are assigned a value of 2.5. This allows the clustering to be most heavily influenced by the strong and medium binding peptides.

In [ ]:
sns.set(font_scale=1.25)

data = p.annotated_predictions.loc[:, (p.df_alleles, 'Rank')]
data[data>2.5] = 2.5

colors = ["#f53b3b", "#bd0b0b", "#08870d", "#026b06", "#00234f", "#1f1f1f"]
nodes = [0, 0.4/2.5, 0.7/2.5, 1.9/2.5, 2.2/2.5, 1]
cmap = LinearSegmentedColormap.from_list("mycmap", list(zip(nodes, colors)))

xticklabels = [x[0] for x in data.columns]

#### Ordered heatmap
Peptides are ordered according to percent rank, starting left-to-right. E.g. First all columns are ordered according to the ranks in the left-most column. Next, all ties (mostly where the left-hand column = 2.5) are ordered according to the next column, and this continues until all columns have been considered.

The colorscale is set such that (approximately) strong binders are represented in red, medium binders in green, and weak bindings fading from dark blue to black. The color scheme is designed to be gray-scale- and colorblind-friendly.

In [ ]:
# make a sorted heatmap
plt.figure(figsize=(12, 8))
g = sns.heatmap(data.sort_values(list(data.columns), ascending=False), cmap=cmap, yticklabels=False,
                xticklabels=xticklabels, vmax=2.5)
g.set_xlabel('')

In [ ]:
'''
#### Clustered heatmap
Peptides are clustered according to binding rank and the results shown in a heatmap. The [linkage method](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html) used is 'ward', and the [distance metric](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.pdist.html) is 'euclidean'.
'''

In [ ]:
'''
# make a clustered heatmap
plt.figure(figsize=(12, 8))
g = sns.clustermap(data.sort_values(list(data.columns)), cmap=cmap, method='complete', metric='euclidean',
                   dendrogram_ratio=(0.15,0.1), yticklabels=False, cbar_pos=(1,0.3,0.05,0.5),
                   xticklabels=xticklabels, vmax=2.5)
g.ax_heatmap.set_xlabel('')
'''